<a href="https://colab.research.google.com/github/HanhengHe/BigDataCourseProj/blob/main/bigdataproj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
RUNLARGE = True
NUMCLASS= 50 if RUNLARGE else 10
dataSize = -1

In [ ]:
# GET FILE

from google.colab import drive
import zipfile

drive.mount('/content/drive/', force_remount=False)

filePath = "/content/drive/MyDrive/Dataset_large" if RUNLARGE else "/content/drive/MyDrive/Dataset_small/data1.zip"

with zipfile.ZipFile(filePath,"r") as zip_ref:
    zip_ref.extractall("targetdir")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data as Data
import torch.optim as optim
import torch.nn.functional as F
from torchvision import models

In [ ]:
# LOADING TRAINING DATA

lsTrainLabel = []
lsTrainData = []

trainingPath = "/content/targetdir/dataset2" if RUNLARGE else "/content/targetdir/data1"

count = 0

with open(trainingPath + "/train_label.txt") as f:
  for line in f.readlines():
    expression = line.split(' ')
    lsTrainLabel.append(int(expression[1]))
    dataRGB = np.load(trainingPath + "/train/" + expression[0]).transpose((2,0,1))
    lsTrainData.append(dataRGB)
    # turn training data into grayscale data
    # lsData.append((dataRGB[:, :, 0] + dataRGB[:, :, 1] + dataRGB[:, :, 2]) / 3)
    count += 1
    if dataSize != -1 and dataSize == count:
        break

INITER = torch.zeros(2, lsTrainData[0].shape[0], lsTrainData[0].shape[1], lsTrainData[0].shape[2])

print("Number of traning data: {0}".format(len(lsTrainLabel)), end='\n')
print("Shape of traning data: {0}".format(lsTrainData[0].shape), end='\n')

Number of traning data: 64163
Shape of traning data: (3, 224, 224)


In [ ]:
trainLabels = np.array(lsTrainLabel)
trainSet = np.array(lsTrainData)

trainSet = torch.from_numpy(trainSet).float()

trainLabels = torch.from_numpy(trainLabels)    # nll
torch_dataset = Data.TensorDataset(trainSet, trainLabels.long())    # nll

# criterion = nn.MSELoss()  # mse
# trainLabels = torch.nn.functional.one_hot(torch.from_numpy(trainLabels), num_classes=NUMCLASS)  # mse
# torch_dataset = Data.TensorDataset(trainSet, trainLabels.float())  # mse

In [ ]:
CPU = torch.device("cpu")
DEVICE = torch.device("cuda:0") if torch.cuda.is_available() else CPU
# DEVICE = CPU
print("DEVICE = %s" % (DEVICE))

DEVICE = cuda:0


In [ ]:
# LOAD TEST DATA

lsTestLabel = []
lsTestData = []

testingPath = "/content/targetdir/dataset2" if RUNLARGE else "/content/targetdir/data1"


with open(testingPath + "/test_label.txt") as f:
  for line in f.readlines():
    expression = line.split(' ')
    lsTestLabel.append(int(expression[1]))
    dataRGB = np.load(testingPath + "/test/" + expression[0]).transpose((2,0,1))
    lsTestData.append(dataRGB)
    # turn training data into grayscale data
    # lsData.append((dataRGB[:, :, 0] + dataRGB[:, :, 1] + dataRGB[:, :, 2]) / 3)

print("Number of testing data: {0}".format(len(lsTestLabel)), end='\n')
print("Shape of testing data: {0}".format(lsTestData[0].shape), end='\n')

Number of testing data: 2500
Shape of testing data: (3, 224, 224)


In [ ]:
class Generator(nn.Module):
    def __init__(self, noiseSize=128, ct_deepth=128):
        super(Generator, self).__init__()
        
        self.model_noise_channel = nn.Sequential(
            nn.ConvTranspose2d(noiseSize, ct_deepth * 8, 4, 1, 0),
            nn.BatchNorm2d(ct_deepth * 8, 0.2),
            nn.LeakyReLU()
            )
        
        self.model_label_channel = nn.Sequential(
            nn.ConvTranspose2d(50, ct_deepth * 8, 4, 1, 0),
            nn.BatchNorm2d(ct_deepth * 8, 0.2),
            nn.LeakyReLU()
            )

        self.model_kernal = nn.Sequential(
            nn.ConvTranspose2d(ct_deepth * 16, ct_deepth * 12, 4, 2, 1),
            nn.BatchNorm2d(ct_deepth * 12, 0.2),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(ct_deepth * 12, ct_deepth * 8, 3, 2, 1),
            nn.BatchNorm2d(ct_deepth * 8, 0.2),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(ct_deepth * 8, ct_deepth * 4, 3, 2, 1),
            nn.BatchNorm2d(ct_deepth * 4, 0.2),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(ct_deepth * 4, ct_deepth * 2, 3, 2, 1),
            nn.BatchNorm2d(ct_deepth * 2, 0.2),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(ct_deepth * 2, ct_deepth, 3, 2, 1),
            nn.BatchNorm2d(ct_deepth, 0.2),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(ct_deepth, 3, 2, 2, 1),
            nn.Tanh()
            )

    def forward(self, x, label):
        x_0 = self.model_noise_channel(x)
        x_1 = self.model_label_channel(label)
        x = torch.cat([x_0, x_1], 1)
        x = self.model_kernal(x)
        return x


class Discriminator(nn.Module):
    def __init__(self, ct_deepth=128):
        super(Discriminator, self).__init__()

        self.model_noise_channel = nn.Sequential(
            nn.Conv2d(3, int(ct_deepth / 2), 4, 1, 2),
            nn.BatchNorm2d(int(ct_deepth / 2), 0.2),
            nn.LeakyReLU()
            )
        
        self.model_label_channel = nn.Sequential(
            nn.Conv2d(50, int(ct_deepth / 2), 4, 1, 2),
            nn.BatchNorm2d(int(ct_deepth / 2), 0.2),
            nn.LeakyReLU()
            )

        self.model_kernal = nn.Sequential(
            nn.Conv2d(ct_deepth, ct_deepth * 2, 3, 2, 1),
            nn.BatchNorm2d(ct_deepth * 2, 0.2),
            nn.LeakyReLU(),
            nn.Conv2d(ct_deepth * 2, ct_deepth * 4, 3, 2, 1),
            nn.BatchNorm2d(ct_deepth * 4, 0.2),
            nn.LeakyReLU(),
            nn.Conv2d(ct_deepth * 4, ct_deepth * 2, 4, 4, 1),
            nn.BatchNorm2d(ct_deepth * 2, 0.2),
            nn.LeakyReLU(),
            nn.Conv2d(ct_deepth * 2, ct_deepth, 4, 4, 1),
            nn.BatchNorm2d(ct_deepth, 0.2),
            nn.LeakyReLU(),
            nn.Conv2d(ct_deepth, 1, 4, 1, 0),
            nn.Sigmoid()
            )

    def forward(self, x, label):
        x_0 = self.model_noise_channel(x)
        x_1 = self.model_label_channel(label)
        x = torch.cat([x_0, x_1], 1)
        x = self.model_kernal(x)
        x = x.view(x.shape[0], -1)
        return x

In [ ]:
# TRAINING MODULE 

import math
from torch.optim.lr_scheduler import StepLR
import time
import os.path
from os import path
import matplotlib.pyplot as plt

class LogHelper:
    def __init__(self):
        self.strLog = ""
    
    def PrintAndLog(self, strMsg: str, show=True, end="\n"):
        self.strLog += strMsg
        self.strLog += end
        if show:
            print(strMsg, end=end)

Epochs = 20
MINLOSS = 1e-5
MINIBATCH_SIZE = 32
LEARNING_RATE_G = 0.001
LEARNING_RATE_D = 0.00002
lrChangeFq_G = 10
lrChangeFq_D = 5
gammaScheduler = 0.79434
outputFrequence = 1
bSaveFile = True
nExtraTrainG_Aid = 5
nExtraTrainG_Normal = 1
dFirstAid = 0.38

logHelper = LogHelper() 

torch.cuda.empty_cache()

loader = Data.DataLoader(
    dataset=torch_dataset,
    batch_size=MINIBATCH_SIZE,
    shuffle=True,
    num_workers=0
)

nSteps = math.ceil(trainLabels.shape[0] / MINIBATCH_SIZE)

model_G = Generator().to(DEVICE)
model_D = Discriminator().to(DEVICE)

# OPTIMIZER
optimizer_G = optim.Adam(model_G.parameters(), lr=LEARNING_RATE_G)
scheduler_G = StepLR(optimizer_G, step_size=lrChangeFq_G, gamma=gammaScheduler)

optimizer_D = optim.Adam(model_D.parameters(), lr=LEARNING_RATE_D)
scheduler_D = StepLR(optimizer_D, step_size=lrChangeFq_D, gamma=gammaScheduler)

criterion = nn.BCELoss()

model_G.train()
model_D.train()

ls_D_Loss = []
ls_G_Loss = []

mOnehot = torch.diag(torch.ones(NUMCLASS))[:, :, None, None]

fill = torch.zeros([NUMCLASS, NUMCLASS, 224, 224])
for i in range(NUMCLASS):
    fill[i, i, :, :] = 1

struct_time = time.localtime() # get struct_time
strTime = time.strftime("%m-%d-%Y_%H:%M:%S", struct_time)

fileName = "CDCGAN_" + strTime
filePath = '/content/drive/MyDrive/BigDataProjLog/CDCGAN/'

for epoch in range(Epochs):

    print("Epoch {0}".format(epoch+1))

    d_CurLoss = 0
    g_CurLoss = 0
    D_x = 0
    for step, (batch_x, batch_y) in enumerate(loader):

        model_D.zero_grad()
        
        # variables used more than once
        curOnehot = mOnehot[batch_y].to(DEVICE)
        noise = torch.randn((batch_x.shape[0], 128, 1, 1)).to(DEVICE)
        filledLabel = fill[batch_y].to(DEVICE)
        realLabel = torch.ones((batch_x.shape[0], 1)).to(DEVICE)

        # train discriminator
        d_ResReal = model_D(batch_x.to(DEVICE), filledLabel)
        d_LossReal = criterion(d_ResReal, realLabel)

        g_Res = model_G(noise, curOnehot)
        d_ResFake = model_D(g_Res, filledLabel)
        D_x += d_ResFake.mean().item() / nSteps
        d_LossFake = criterion(d_ResFake, torch.zeros((batch_x.shape[0], 1)).to(DEVICE))

        d_Loss = d_LossReal + d_LossFake
        
        bNeedFirstAid = d_ResFake.mean().item() < dFirstAid
        if not bNeedFirstAid:
            d_Loss.backward()
            optimizer_D.step()

        d_CurLoss += d_Loss.item() / nSteps

        curExtraTrainG = nExtraTrainG_Aid if bNeedFirstAid else nExtraTrainG_Normal

        # train generator
        for _ in range(curExtraTrainG):
            model_G.zero_grad()

            g_Res = model_G(noise, curOnehot)
            d_ResTrainG = model_D(g_Res, filledLabel)
            d_LossTrainG = criterion(d_ResTrainG, realLabel)  # as real

            d_LossTrainG.backward()
            optimizer_G.step()

            g_CurLoss += d_LossTrainG.item() / nSteps / curExtraTrainG
        
        # print("     [{0}|{1}] G loss: {2:.4}, D loss: {3:.4}, D(x): {4:.4}".format(step+1, nSteps, d_LossTrainG.item(), d_Loss.item(), d_ResFake.mean().item()))

    scheduler_D.step()
    LEARNING_RATE_D = optimizer_D.param_groups[0]['lr']

    scheduler_G.step()
    LEARNING_RATE_G = optimizer_G.param_groups[0]['lr']

    ls_G_Loss.append(g_CurLoss)
    ls_D_Loss.append(d_CurLoss)

    lrMsg = " lr_G = {0:.5}, lr_D = {1:.5}".format(LEARNING_RATE_G, LEARNING_RATE_D)
    curMsg = " Epoch {0}, G loss: {1:.4}, D loss: {2:.4}, D(x): {3:.4}".format(epoch+1, g_CurLoss, d_CurLoss, D_x) + lrMsg

    logHelper.PrintAndLog(curMsg, show=((epoch == 0) or ((epoch + 1) % outputFrequence == 0)))

    if bSaveFile: 
        torch.save(model_G.state_dict(), filePath + fileName + "Generator_E{0}_GL{1}.pth".format(epoch+1, g_CurLoss))
        torch.save(model_D.state_dict(), filePath + fileName + "Disc_E{0}_GL{1}_DX{2}.pth".format(epoch+1, d_CurLoss, D_x))

if bSaveFile: 
    if not os.path.exists(filePath):
        os.makedirs(filePath)

    with open(filePath + fileName + '.log', 'w') as f:
        f.write(logHelper.strLog)

if bSaveFile: 
    torch.save(model_G.state_dict(), filePath + fileName + "Generator.pth")
    torch.save(model_D.state_dict(), filePath + fileName + "Disc.pth")

# plot
plt.figure()
plt.xlabel('epoch')
plt.ylabel('Loss')
plt.xticks([])
plt_D, = plt.plot(list(range(0,len(ls_D_Loss))), ls_D_Loss)
plt_G, = plt.plot(list(range(0,len(ls_G_Loss))), ls_G_Loss,color='red',linewidth=1.0,linestyle='--')
plt.legend(handles=[plt_D,plt_G],labels=['Loss D','Loss G'],loc='best')
plt.title(fileName)
if bSaveFile: 
    plt.savefig(filePath + fileName + '.png')
plt.show()

Epoch 1
 Epoch 1, G loss: 1.117, D loss: 0.7789, D(x): 0.3537 lr_G = 0.001, lr_D = 2e-05
Epoch 2
 Epoch 2, G loss: 1.318, D loss: 0.5825, D(x): 0.2787 lr_G = 0.001, lr_D = 2e-05
Epoch 3
 Epoch 3, G loss: 1.332, D loss: 0.5918, D(x): 0.2773 lr_G = 0.001, lr_D = 2e-05
Epoch 4
 Epoch 4, G loss: 1.315, D loss: 0.6011, D(x): 0.2823 lr_G = 0.001, lr_D = 2e-05
Epoch 5
 Epoch 5, G loss: 1.313, D loss: 0.6059, D(x): 0.2838 lr_G = 0.001, lr_D = 1.5887e-05
Epoch 6
 Epoch 6, G loss: 1.348, D loss: 0.5979, D(x): 0.2748 lr_G = 0.001, lr_D = 1.5887e-05
Epoch 7
 Epoch 7, G loss: 1.338, D loss: 0.6038, D(x): 0.2781 lr_G = 0.001, lr_D = 1.5887e-05
Epoch 8


In [ ]:
from torchvision.models.resnet import ResNet50_Weights

class ResNet18(nn.Module):
    def __init__(self):
        super(ResNet18, self).__init__()
        self.model = models.resnet18()

    def forward(self, x):
        x = self.model(x)
        x = x.view(x.size(0), -1)
        return x


class ResNet34(nn.Module):
    def __init__(self):
        super(ResNet34, self).__init__()
        self.model = models.resnet34()

    def forward(self, x):
        x = self.model(x)
        x = x.view(x.size(0), -1)
        return x


class ResNet50(nn.Module):
    def __init__(self):
        super(ResNet50, self).__init__()
        self.model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)

    def forward(self, x):
        x = self.model(x)
        x = x.view(x.size(0), -1)
        return x


class ResNet101(nn.Module):
    def __init__(self):
        super(ResNet101, self).__init__()
        self.model = models.resnet101()

    def forward(self, x):
        x = self.model(x)
        x = x.view(x.size(0), -1)
        return x


class ResNet152(nn.Module):
    def __init__(self):
        super(ResNet152, self).__init__()
        self.model = models.resnet152()

    def forward(self, x):
        x = self.model(x)
        x = x.view(x.size(0), -1)
        return x

In [ ]:
from torchvision.models.resnet import ResNeXt50_32X4D_Weights, ResNeXt101_32X8D_Weights

class ResNeXt50(nn.Module):
    def __init__(self):
        super(ResNeXt50, self).__init__()
        self.model = models.resnext50_32x4d(weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V2)

    def forward(self, x):
        x = self.model(x)
        x = x.view(x.size(0), -1)
        return x

class ResNeXt101_32(nn.Module):
    def __init__(self):
        super(ResNeXt101_32, self).__init__()
        self.model = models.resnext101_32x8d(weights=ResNeXt101_32X8D_Weights.IMAGENET1K_V1)

    def forward(self, x):
        x = self.model(x)
        x = x.view(x.size(0), -1)
        return x

class ResNeXt101_64(nn.Module):
    def __init__(self):
        super(ResNeXt101_64, self).__init__()
        self.model = models.resnext101_64x4d()

    def forward(self, x):
        x = self.model(x)
        x = x.view(x.size(0), -1)
        return x

In [ ]:
from torchvision.models.densenet import DenseNet121_Weights, DenseNet161_Weights

class DenseNet121(nn.Module):
    def __init__(self):
        super(DenseNet121, self).__init__()
        self.model = models.densenet121(weights=DenseNet121_Weights.IMAGENET1K_V1)

    def forward(self, x):
         x = self.model(x)
         x = x.view(x.size(0), -1)
         return x

class DenseNet161(nn.Module):
    def __init__(self):
        super(DenseNet161, self).__init__()
        self.model = models.densenet161(weights=DenseNet161_Weights.IMAGENET1K_V1)

    def forward(self, x):
         x = self.model(x)
         x = x.view(x.size(0), -1)
         return x

class DenseNet169(nn.Module):
    def __init__(self):
        super(DenseNet169, self).__init__()
        self.model = models.densenet169()

    def forward(self, x):
         x = self.model(x)
         x = x.view(x.size(0), -1)
         return x

class DenseNet201(nn.Module):
    def __init__(self):
        super(DenseNet201, self).__init__()
        self.model = models.densenet201()

    def forward(self, x):
         x = self.model(x)
         x = x.view(x.size(0), -1)
         return x

In [ ]:
from torchvision.models.efficientnet import EfficientNet_V2_S_Weights
from torchvision.models.convnext import ConvNeXt_Base_Weights

class EfficientNetV2(nn.Module):
    def __init__(self):
        super(EfficientNetV2, self).__init__()
        self.model = models.efficientnet_v2_s(weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1)

    def forward(self, x):
         x = self.model(x)
         x = x.view(x.size(0), -1)
         return x

class ConvNeXt_base(nn.Module):
    def __init__(self):
        super(ConvNeXt_base, self).__init__()
        self.model = models.convnext_base(weights=ConvNeXt_Base_Weights.IMAGENET1K_V1)

    def forward(self, x):
         x = self.model(x)
         x = x.view(x.size(0), -1)
         return x

class ConvNeXt_large(nn.Module):
    def __init__(self):
        super(ConvNeXt_large, self).__init__()
        self.model = models.convnext_large()

    def forward(self, x):
         x = self.model(x)
         x = x.view(x.size(0), -1)
         return x

class MaxVit(nn.Module):
    def __init__(self):
        super(MaxVit, self).__init__()
        self.model = models.maxvit()

    def forward(self, x):
         x = self.model(x)
         x = x.view(x.size(0), -1)
         return x

In [ ]:
network_dict = {"ResNet18": ResNet18,
                "ResNet34": ResNet34,
                "ResNet50": ResNet50,
                "ResNet101": ResNet101,
                "ResNet152": ResNet152,

                "DenseNet121": DenseNet121,
                "DenseNet161": DenseNet161,
                "DenseNet169": DenseNet169,
                "DenseNet201": DenseNet201,

                "ResNeXt50": ResNeXt50,
                "ResNeXt101_32": ResNeXt101_32,
                "ResNeXt101_64": ResNeXt101_64,

                "EfficientNetV2": EfficientNetV2,

                "ConvNeXt_base": ConvNeXt_base,
                "ConvNeXt_large": ConvNeXt_large}

In [ ]:
# NETWORK DEFINE MODULE

torch.cuda._initialized = True

class Network(nn.Module):
    
    def __init__(self, initer: torch.Tensor, num_classes: int, kernalType: str):
        
        super(Network, self).__init__()
        self.backbone = network_dict[kernalType]()
        self.fc = nn.Linear(self.backbone(initer).shape[1], num_classes)
        
    def forward(self, x):
        
        x = self.backbone(x)
        x = self.fc(x)
        # x = F.softmax(x, dim=1)  # note this when use ce
        return x 

In [ ]:
# TRAINING MODULE 

import math
from torch.optim.lr_scheduler import StepLR
import time
import os.path
from os import path
import matplotlib.pyplot as plt

class LogHelper:
    def __init__(self):
        self.strLog = ""
    
    def PrintAndLog(self, strMsg: str, show=True, end="\n"):
        self.strLog += strMsg
        self.strLog += end
        if show:
            print(strMsg, end=end)

def Processing(KERNALTYPE, 
               Epochs = 50, 
               MINLOSS = 1e-5, 
               MINIBATCH_SIZE = 128, 
               LEARNING_RATE = 0.001, 
               lr_diff = 1e-5, 
               lrChangeFq = -1,
               gammaScheduler = 0.8,
               outputFrequence = 10
               ):

    lsTrainLoss = []

    logHelper = LogHelper() 

    bChangeLr = lrChangeFq != -1
    torch.cuda.empty_cache()
    model = Network(INITER, NUMCLASS, KERNALTYPE).to(DEVICE)

    # put the dataset into DataLoader
    loader = Data.DataLoader(
        dataset=torch_dataset,
        batch_size=MINIBATCH_SIZE,
        shuffle=True,
        num_workers=0
    )

    nSteps = math.ceil(trainLabels.shape[0] / MINIBATCH_SIZE)

    criterion = nn.CrossEntropyLoss() # ce
    # criterion = nn.NLLLoss()  # nll

    # OPTIMIZER
    # optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9)
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    scheduler = StepLR(optimizer, step_size=lrChangeFq if bChangeLr else Epochs, gamma=gammaScheduler)

    logHelper.PrintAndLog("\n\nKERBALTYPE: " + KERNALTYPE)
    logHelper.PrintAndLog("INIT LEARNING RATE={0}; EPOCH={1}; CHGFR={2}; GAMMA={3}".format(LEARNING_RATE, Epochs, lrChangeFq, gammaScheduler))
    logHelper.PrintAndLog("TRAINING...")

    model.train()
    lastLoss = 0
    for epoch in range(Epochs):

        curLoss = 0

        for step, (batch_x, batch_y) in enumerate(loader):
            optimizer.zero_grad()        
            output = model(batch_x.to(DEVICE))
            loss = criterion(output, batch_y.to(DEVICE))  # ce
            loss.backward()
            optimizer.step()

            curLoss += loss.item() / nSteps

        lsTrainLoss.append(curLoss)

        if bChangeLr:
            scheduler.step()
            LEARNING_RATE = optimizer.param_groups[0]['lr']
        
        lrMsg = " lr = {0:.5}".format(LEARNING_RATE) if bChangeLr else ""
        curMsg = " Epoch {0}, general loss: {1}".format(epoch+1, curLoss) + lrMsg

        logHelper.PrintAndLog(curMsg, show=((epoch == 0) or ((epoch + 1) % outputFrequence == 0)))

        if curLoss <= MINLOSS:
            logHelper.PrintAndLog(" Terminated with low loss. Epoch {0}, general loss: {1}".format(epoch+1, curLoss) + lrMsg)
            break
        
        if abs(lastLoss - curLoss) <= lr_diff:
            logHelper.PrintAndLog(" Terminated with unchanged loss. Epoch {0}, general loss: {1}".format(epoch+1, curLoss) + lrMsg)
            break
        
        lastLoss = curLoss

    # TEST MODULE

    testLabels = np.array(lsTestLabel)
    testSet = np.array(lsTestData)

    testSet = torch.from_numpy(testSet).float()
    testLabels = torch.from_numpy(testLabels).to(DEVICE)

    model.eval()
    logHelper.PrintAndLog("TESTING...")

    test_loss = 0
    correct = 0

    # first transform the data to dataset can be processed by torch
    # torch_dataset = Data.TensorDataset(testSet, testLabels.long())  # nll

    testLabelsOneHot = torch.nn.functional.one_hot(testLabels, num_classes=NUMCLASS)  # mse
    torch_dataset = Data.TensorDataset(testSet, testLabelsOneHot.float())  # mse

    # put the dataset into DataLoader
    loader = Data.DataLoader(
        dataset=torch_dataset,
        batch_size=MINIBATCH_SIZE,
        shuffle=False,
        num_workers=0
    )

    with torch.no_grad():
        for step, (batch_x, batch_y) in enumerate(loader):
            output = model(batch_x.to(DEVICE))
            test_loss += criterion(output, batch_y.to(DEVICE)).item()  # mse
            predict = output.max(1, keepdim=True)[1]  # find the prediction
            labels = batch_y.max(1, keepdim=True)[1]
            correct += predict.eq(labels.view_as(predict)).sum().item()

    testSize = testLabels.shape[0]
    test_loss /= testSize
    curRate = correct / testSize
    logHelper.PrintAndLog(" Test: Average loss:%s, Accuracy: %s/%s (%s)"
        % (test_loss, correct, testSize, curRate))
    
    struct_time = time.localtime() # get struct_time
    strTime = time.strftime("%m-%d-%Y_%H:%M:%S", struct_time)

    fileName = KERNALTYPE + ("_L_" if RUNLARGE else "_S_") + strTime
    filePath = '/content/drive/MyDrive/BigDataProjLog/{0}/'.format(KERNALTYPE)

    if not os.path.exists(filePath):
        os.makedirs(filePath)

    with open(filePath + fileName + '.log', 'w') as f:
        f.write(logHelper.strLog)
    
    torch.save(model.state_dict(), filePath + fileName + "_ACC={0:.4}.pth".format(curRate))

    # plot
    plt.figure()
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.xticks([])
    plt.plot(list(range(0,len(lsTrainLoss))), lsTrainLoss)
    plt.title(fileName)
    plt.savefig(filePath + fileName + '.png')
    plt.show()

In [ ]:
# test
"""for key in network_dict:
    Processing(key)"""

'for key in network_dict:\n    Processing(key)'

In [ ]:
"""
"ResNet18", "ResNet34", "ResNet50", "ResNet101", "ResNet152", 
"DenseNet121", "DenseNet161", "DenseNet169", "DenseNet201", 
"ResNeXt50", "ResNeXt101_32", "ResNeXt101_64", "EfficientNetV2", 
"ConvNeXt_base", "ConvNeXt_large"
"""

'\n"ResNet18", "ResNet34", "ResNet50", "ResNet101", "ResNet152", \n"DenseNet121", "DenseNet161", "DenseNet169", "DenseNet201", \n"ResNeXt50", "ResNeXt101_32", "ResNeXt101_64", "EfficientNetV2", \n"ConvNeXt_base", "ConvNeXt_large"\n'

In [ ]:
Processing("ResNet18", lrChangeFq=5, outputFrequence=10)

In [ ]:
from torchsummary import summary
model = Network(INITER, NUMCLASS, "Vit").to(DEVICE)
summary(model, (3, 224, 224))